In [0]:
%run "./02 Setup Data Sets to perform joins"

+---------+--------------------+-------------------+---------+-------------------+
course_id| course_title|course_published_dt|is_active| last_updated_ts|
+---------+--------------------+-------------------+---------+-------------------+
 1| Mastering Python| 2021-01-14| true|2021-02-18 16:57:25|
 2|Data Engineering ...| 2021-02-10| true|2021-03-05 12:07:33|
 3| Mastering Pyspark| 2021-01-07| true|2021-04-06 10:05:42|
 4| AWS Essentials| 2021-03-19| false|2021-04-10 02:25:36|
 5| Docker 101| 2021-02-28| true|2021-03-21 07:18:52|
+---------+--------------------+-------------------+---------+-------------------+

+-------+---------------+--------------+--------------------+
user_id|user_first_name|user_last_name| user_email|
+-------+---------------+--------------+--------------------+
 1| Sandra| Karpov| skarpov0@ovh.net|
 2| Kari| Dearth|kdearth1@so-net.n...|
 3| Joanna| Spennock|jspennock2@redcro...|
 4| Hirsch| Conaboy|hconaboy3@barnesa...|
 5| Loreen| Malin|lmalin4@independe...|
 6| Augy| Christon| achriston5@mlb.com|
 7| Trudey| Choupin| tchoupin6@de.vu|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com|
 9| Vassily| Tamas|vtamas8@businessw...|
 10| Wells| Simpkins|wsimpkins9@amazon...|
+-------+---------------+--------------+--------------------+

+-------------------+-------+---------+----------+
course_enrolment_id|user_id|course_id|price_paid|
+-------------------+-------+---------+----------+
 1| 10| 2| 9.99|
 2| 5| 2| 9.99|
 3| 7| 5| 10.99|
 4| 9| 2| 9.99|
 5| 8| 2| 9.99|
 6| 5| 5| 10.99|
 7| 4| 5| 10.99|
 8| 7| 3| 10.99|
 9| 8| 5| 10.99|
 10| 3| 3| 10.99|
 11| 7| 5| 10.99|
 12| 3| 2| 9.99|
 13| 5| 2| 9.99|
 14| 4| 3| 10.99|
 15| 8| 2| 9.99|
+-------------------+-------+---------+----------+

In [0]:
help(courses_df.join)

Help on method join in module pyspark.sql.dataframe:

join(other, on=None, how=None) method of pyspark.sql.dataframe.DataFrame instance
 Joins with another :class:`DataFrame`, using the given join expression.
 
 .. versionadded:: 1.3.0
 
 Parameters
 ----------
 other : :class:`DataFrame`
 Right side of the join
 on : str, list or :class:`Column`, optional
 a string for the join column name, a list of column names,
 a join expression (Column), or a list of Columns.
 If `on` is a string or a list of strings indicating the name of the join column(s),
 the column(s) must exist on both sides, and this performs an equi-join.
 how : str, optional
 default ``inner``. Must be one of: ``inner``, ``cross``, ``outer``,
 ``full``, ``fullouter``, ``full_outer``, ``left``, ``leftouter``, ``left_outer``,
 ``right``, ``rightouter``, ``right_outer``, ``semi``, ``leftsemi``, ``left_semi``,
 ``anti``, ``leftanti`` and ``left_anti``.
 
 Examples
 --------
 The following performs a full outer join between ``df1`` and ``df2``.
 
 >>> from pyspark.sql.functions import desc
 >>> df.join(df2, df.name == df2.name, 'outer').select(df.name, df2.height) .sort(desc("name")).collect()
 [Row(name='Bob', height=85), Row(name='Alice', height=None), Row(name=None, height=80)]
 
 >>> df.join(df2, 'name', 'outer').select('name', 'height').sort(desc("name")).collect()
 [Row(name='Tom', height=80), Row(name='Bob', height=85), Row(name='Alice', height=None)]
 
 >>> cond = [df.name == df3.name, df.age == df3.age]
 >>> df.join(df3, cond, 'outer').select(df.name, df3.age).collect()
 [Row(name='Alice', age=2), Row(name='Bob', age=5)]
 
 >>> df.join(df2, 'name').select(df.name, df2.height).collect()
 [Row(name='Bob', height=85)]
 
 >>> df.join(df4, ['name', 'age']).select(df.name, df.age).collect()
 [Row(name='Bob', age=5)]

* Get all the user details along with course enrolment details (if the user have any course enrolments).
* If the users does not have any course enrolments, we need to get all user details. Course details will be substituted with null values.
  * Need to perform left or right outer join between **users_df** and **course_enrolments_df**.
  * We will use left for this lecture. As `users_df` is from parent table and as we are going to use **left outer join**, we need to invoke `join` on top of `users_df`.
  * Here are the fields that needs to be displayed.
    * All fields from `users_df`
    * `course_id` and `course_enrolment_id` from `course_enrolments_df`
* For this example using these data frames, using just `outer` also give same results. But it is not correct to use `outer`.
* `how='outer'` means **full outer join**.

In [0]:
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left'). \
    show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
user_id|user_first_name|user_last_name| user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
 1| Sandra| Karpov| skarpov0@ovh.net| null| null| null| null|
 2| Kari| Dearth|kdearth1@so-net.n...| null| null| null| null|
 3| Joanna| Spennock|jspennock2@redcro...| 10| 3| 3| 10.99|
 3| Joanna| Spennock|jspennock2@redcro...| 12| 3| 2| 9.99|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 7| 4| 5| 10.99|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 14| 4| 3| 10.99|
 5| Loreen| Malin|lmalin4@independe...| 2| 5| 2| 9.99|
 5| Loreen| Malin|lmalin4@independe...| 6| 5| 5| 10.99|
 5| Loreen| Malin|lmalin4@independe...| 13| 5| 2| 9.99|
 6| Augy| Christon| achriston5@mlb.com| null| null| null| null|
 7| Trudey| Choupin| tchoupin6@de.vu| 3| 7| 5| 10.99|
 7| Trudey| Choupin| tchoupin6@de.vu| 8| 7| 3| 10.99|
 7| Trudey| Choupin| tchoupin6@de.vu| 11| 7| 5| 10.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 5| 8| 2| 9.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 9| 8| 5| 10.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 15| 8| 2| 9.99|
 9| Vassily| Tamas|vtamas8@businessw...| 4| 9| 2| 9.99|
 10| Wells| Simpkins|wsimpkins9@amazon...| 1| 10| 2| 9.99|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+

In [0]:
# left or left_outer or leftouter are same.

users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left_outer'). \
    show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
user_id|user_first_name|user_last_name| user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
 1| Sandra| Karpov| skarpov0@ovh.net| null| null| null| null|
 2| Kari| Dearth|kdearth1@so-net.n...| null| null| null| null|
 3| Joanna| Spennock|jspennock2@redcro...| 10| 3| 3| 10.99|
 3| Joanna| Spennock|jspennock2@redcro...| 12| 3| 2| 9.99|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 7| 4| 5| 10.99|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 14| 4| 3| 10.99|
 5| Loreen| Malin|lmalin4@independe...| 2| 5| 2| 9.99|
 5| Loreen| Malin|lmalin4@independe...| 6| 5| 5| 10.99|
 5| Loreen| Malin|lmalin4@independe...| 13| 5| 2| 9.99|
 6| Augy| Christon| achriston5@mlb.com| null| null| null| null|
 7| Trudey| Choupin| tchoupin6@de.vu| 3| 7| 5| 10.99|
 7| Trudey| Choupin| tchoupin6@de.vu| 8| 7| 3| 10.99|
 7| Trudey| Choupin| tchoupin6@de.vu| 11| 7| 5| 10.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 5| 8| 2| 9.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 9| 8| 5| 10.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 15| 8| 2| 9.99|
 9| Vassily| Tamas|vtamas8@businessw...| 4| 9| 2| 9.99|
 10| Wells| Simpkins|wsimpkins9@amazon...| 1| 10| 2| 9.99|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+

In [0]:
users_df. \
    join(course_enrolments_df, 'user_id', 'left'). \
    show()

+-------+---------------+--------------+--------------------+-------------------+---------+----------+
user_id|user_first_name|user_last_name| user_email|course_enrolment_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+---------+----------+
 1| Sandra| Karpov| skarpov0@ovh.net| null| null| null|
 2| Kari| Dearth|kdearth1@so-net.n...| null| null| null|
 3| Joanna| Spennock|jspennock2@redcro...| 10| 3| 10.99|
 3| Joanna| Spennock|jspennock2@redcro...| 12| 2| 9.99|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 7| 5| 10.99|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 14| 3| 10.99|
 5| Loreen| Malin|lmalin4@independe...| 2| 2| 9.99|
 5| Loreen| Malin|lmalin4@independe...| 6| 5| 10.99|
 5| Loreen| Malin|lmalin4@independe...| 13| 2| 9.99|
 6| Augy| Christon| achriston5@mlb.com| null| null| null|
 7| Trudey| Choupin| tchoupin6@de.vu| 3| 5| 10.99|
 7| Trudey| Choupin| tchoupin6@de.vu| 8| 3| 10.99|
 7| Trudey| Choupin| tchoupin6@de.vu| 11| 5| 10.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 5| 2| 9.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 9| 5| 10.99|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 15| 2| 9.99|
 9| Vassily| Tamas|vtamas8@businessw...| 4| 2| 9.99|
 10| Wells| Simpkins|wsimpkins9@amazon...| 1| 2| 9.99|
+-------+---------------+--------------+--------------------+-------------------+---------+----------+

In [0]:
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left'). \
    select(users_df['*'], course_enrolments_df['course_id'], course_enrolments_df['course_enrolment_id']). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
user_id|user_first_name|user_last_name| user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
 1| Sandra| Karpov| skarpov0@ovh.net| null| null|
 2| Kari| Dearth|kdearth1@so-net.n...| null| null|
 3| Joanna| Spennock|jspennock2@redcro...| 3| 10|
 3| Joanna| Spennock|jspennock2@redcro...| 2| 12|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 5| 7|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 3| 14|
 5| Loreen| Malin|lmalin4@independe...| 2| 2|
 5| Loreen| Malin|lmalin4@independe...| 5| 6|
 5| Loreen| Malin|lmalin4@independe...| 2| 13|
 6| Augy| Christon| achriston5@mlb.com| null| null|
 7| Trudey| Choupin| tchoupin6@de.vu| 5| 3|
 7| Trudey| Choupin| tchoupin6@de.vu| 3| 8|
 7| Trudey| Choupin| tchoupin6@de.vu| 5| 11|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 2| 5|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 5| 9|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 2| 15|
 9| Vassily| Tamas|vtamas8@businessw...| 2| 4|
 10| Wells| Simpkins|wsimpkins9@amazon...| 2| 1|
+-------+---------------+--------------+--------------------+---------+-------------------+

In [0]:
# using alias
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id, 'left'). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
user_id|user_first_name|user_last_name| user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
 1| Sandra| Karpov| skarpov0@ovh.net| null| null|
 2| Kari| Dearth|kdearth1@so-net.n...| null| null|
 3| Joanna| Spennock|jspennock2@redcro...| 3| 10|
 3| Joanna| Spennock|jspennock2@redcro...| 2| 12|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 5| 7|
 4| Hirsch| Conaboy|hconaboy3@barnesa...| 3| 14|
 5| Loreen| Malin|lmalin4@independe...| 2| 2|
 5| Loreen| Malin|lmalin4@independe...| 5| 6|
 5| Loreen| Malin|lmalin4@independe...| 2| 13|
 6| Augy| Christon| achriston5@mlb.com| null| null|
 7| Trudey| Choupin| tchoupin6@de.vu| 5| 3|
 7| Trudey| Choupin| tchoupin6@de.vu| 3| 8|
 7| Trudey| Choupin| tchoupin6@de.vu| 5| 11|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 2| 5|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 5| 9|
 8| Nadine| Grimsdell|ngrimsdell7@sohu.com| 2| 15|
 9| Vassily| Tamas|vtamas8@businessw...| 2| 4|
 10| Wells| Simpkins|wsimpkins9@amazon...| 2| 1|
+-------+---------------+--------------+--------------------+---------+-------------------+

In [0]:
# Get all the users who have not enroled for any courses
# Recommended to use primary key in the child table when comparing with null values

users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id, 'left'). \
    filter('ce.course_enrolment_id IS NULL'). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
user_id|user_first_name|user_last_name| user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
 1| Sandra| Karpov| skarpov0@ovh.net| null| null|
 2| Kari| Dearth|kdearth1@so-net.n...| null| null|
 6| Augy| Christon| achriston5@mlb.com| null| null|
+-------+---------------+--------------+--------------------+---------+-------------------+

* Get number of courses enroled by each user
* If there are no enrolments, then count should return 0

In [0]:
# count will give incorrect results
# Even though users 1, 2, 6 are not enrolled for any courses, it returns 1
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id, 'left'). \
    groupBy('u.user_id'). \
    count(). \
    orderBy('u.user_id'). \
    show()

+-------+-----+
user_id|count|
+-------+-----+
 1| 1|
 2| 1|
 3| 2|
 4| 2|
 5| 3|
 6| 1|
 7| 3|
 8| 3|
 9| 1|
 10| 1|
+-------+-----+

In [0]:
from pyspark.sql.functions import sum, when

In [0]:
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id, 'left'). \
    groupBy('u.user_id'). \
    agg(sum(when(course_enrolments_df['course_enrolment_id'].isNull(), 0).otherwise(1)).alias('course_count')). \
    orderBy('u.user_id'). \
    show()

+-------+------------+
user_id|course_count|
+-------+------------+
 1| 0|
 2| 0|
 3| 2|
 4| 2|
 5| 3|
 6| 0|
 7| 3|
 8| 3|
 9| 1|
 10| 1|
+-------+------------+

In [0]:
from pyspark.sql.functions import expr

In [0]:
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id, 'left'). \
    groupBy('u.user_id'). \
    agg(sum(expr('''
        CASE WHEN ce.course_enrolment_id IS NULL
            THEN 0
        ELSE 1
        END
    ''')).alias('course_count')). \
    orderBy('u.user_id'). \
    show()

+-------+------------+
user_id|course_count|
+-------+------------+
 1| 0|
 2| 0|
 3| 2|
 4| 2|
 5| 3|
 6| 0|
 7| 3|
 8| 3|
 9| 1|
 10| 1|
+-------+------------+